# Topology of Deep Neural Networks

## 1. Introduction

Analyze the topology of deep neural nets in binary classification tasks:  
1) How is a simple neural net built?  
2) Binary classification  
3) Topology of neural nets  
4) Analysis of Topology

### 1.1. Neural Nets

A neural net is given by the composition of functions of the form $f(x) = K(Wx + b)$, together with some scoring function.  
With $K$ being some non-linear activation function, the weight matrix $W$ and a bias vector $b$.  
In the picture below, each hidden layer corresponds to one of these functions.

![alt text](images/intro_2.png "Simple neural net")

### 1.2. Binary Classification

In classification tasks, we try to say to which of the classes of a given set, a picture corresponds.  
In binary classification, we have only two classes.

### 1.3. Topology of neural nets

The figure below gives an inuition of what is meant by the topology of a neural net

![alt text](images/intro_1.png "Title")

- Given two disjoint manifolds (green and red)
- Each corresponding to a certain class (e.g. red are all cat images and green are all dog images in our train set)
- Each step corresponds to one layer in a well trained neural net
- The betti numbers change in the following way:  
$\beta~(red): (1,2,0) \rightarrow (1,2,0) \rightarrow (2,1,0) \rightarrow (2,0,0) \rightarrow (1,0,0) \rightarrow (1,0,0)$  
$\beta~(green): (2,2,0) \rightarrow (2,2,0) \rightarrow (2,1,0) \rightarrow (2,0,0) \rightarrow (2,0,0) \rightarrow (1,0,0)$  
- In the end we get two disjoint balls

### 1.4. Analysis of Topology

The main part of our presentation will be about analyzing the topology of some datasets in different layers of deep neural nets and how they change with respect to different activation functions.  
This will give a nice inutition of how neural nets work and why you should use certain activation functions and some others not.  
For this, we will not focus on non-topological problems, like the vanishing gradient problem.

### 1.5 Key questions and findings

The results, that we will present will give an intuition, to two peculiar mysteries in Deep Learning:
- The ReLU activation function generally performs better, than other activation functions, like the Sigmoid or the Hyperbolic Tangent.
- Despite the fact, that shallow networks are able to approximate any function arbitrarily well, deeper networks (when trained well, which is not easy) perform better than shallower networks

## 2. Methodology

- We seek to classify two disjoint manifolds $M_a, M_b \subset \mathbb{R}^d$.  
- Sample large but finite set of points $T\subset M_a \cup M_b$ uniformly and densely. Write $T_i = T \cap M_i, i \in \{a,b\}$.  
- Feedforward NN is given by composition $\nu = s \circ f_l \circ f_{l-1} \circ \dots \circ f_2 \circ f_1$, where the $f_i$ are the layers of the NN and $s$ is the score function.  
- Write $\nu_j = f_j \circ \dots \circ f_2 \circ f_1$ to denote the first $j$ layers of the NN.  
- Train the network until it correctly classifies all training examples and almost all test examples. We call such a network "well-trained".  
- Experiments are intended to show the topologies of $\nu_j(M_a)$ and $\nu_j(M_b)$ as j runs from 1 to $l$, for different manifolds and network architectures.  
- Perform experiments on both simulated datasets, where we know the topology in advance, and real-world data. Real world datasets are more difficult to handle for various reasons, but the most important one for us is, that they have extremely complex topologies in general.  
- Thus the experiments on simulated datasets are very extensive and we can then use some real-world datasets to validate our findings.

### (i) Generate the simulated datasets

![alt text](images/simulated_data_1.png "Title")

Three simulated data sets $D-\mathrm{I}$, $D-\mathrm{II}$, $D-\mathrm{III}$:  
$D-\mathrm{I}$ is sampled from a red two dimensional disk ($M_{b}$) with $9$ green disks positioned in it ($M_{a}$).  
We have the betti numbers $\beta~(M_{a}) = (9,0)$ and $\beta~(M_{b}) = (1,9)$.  
$D-\mathrm{II}$ is sampled from a 3D manifold consisting of $9$ intrlocked solid tori.  
We have the betti numbers $\beta~(M_{a}) = (9,9,0)$ and $\beta~(M_{b}) = (9,9,0)$.  
$D-\mathrm{III}$ is sampled from a 3D manifold consisting of $9$ red spheres that each enclose a smaller green sphere that each enclose a red ball.  
We have the betti numbers $\beta~(M_{a}) = (9,0,9)$ and $\beta~(M_{b}) = (18,0,9)$.  

Each Manifold is sampled uniformly and densely.

### (ii) Training neural networks

Want to examine topology changing effects of:
- different activations (ReLU, leaky ReLU ($\alpha = 0.2$), tanh)
- different network depths (4 to 10 layers)
- different network widths (6 to 50 neurons per layer)

We use softmax as scoring function and every network is trained to zero training error and near zero (~0.01%) test error.

Below is a figure showing all the network architectures.

![alt text](images/training_1.png "Title")

As an important note on the activation functions:  
- ReLU and leakyReLU are both non-homeomorphic maps, but tanh is  
- homeomorphic maps cannot reduce betti numbers since they dont't change the topology  
- The only reason tanh can actually change the topology, is because in finite-precision arithmetic, tanh can actually take the values -1 and 1 and hence it is not a homeomorphism anymore (artanh is not defined for -1 and 1)

### (iii) Computing Homology

Building the Vietoris-Rips complex.

We don't simply use the Euclidean distance to build the VR-complex. Instead we first build the k-nearest-neighbour graph and use the geodesic distance on it, denoted by $\delta_k$. For each $x_i, x_j \in X$ the distance $\delta_k(x_i, x_j)$ is defined by the minimal number of edges between them in the k-nearest neighbour graph. This has the effect of normalizing distances across layers in neural networks, while preserving connectivity of nearest neighbors. This is desirable, because the layers will change geometry quite drastically and this metric is rather robust to geometric changes, but will still reveal the topological ones, that we are looking for.

Now, our Vietoris-Rips complex depends on two parameters, that we need to set: $k$ for the metric and the usual $\epsilon$ we need for the VR-complex.

For simulated data we will set parameters, such that the corresponding VR-complex has the same homology as the simulated dataset and then keep these parameters throughout the layers in the neural networks. This means, that we do not have to do persistent homology in every layer of every network, which will save us a lot of time, that we can invest in more experiments.

First set $\epsilon = 1$ and find a $k$, such that the first betti-number is correct, then fix the found $k_\star$ and proceed to tweak $\epsilon$, until all the betti-numbers are correct.

The authors give the following rough time-estimates: The time required to compute even a single Betti-number can be as much as 30 minutes for the output of a $50$-neuron wide layer (i.e. a $50$-dimensional point cloud), while the single computation of persistent homology to obtain $k_\star$ and $\epsilon_\star$ takes about 80 minutes.

The resulting numbers for our simulated data sets are:
- For $D-\mathrm{I}$ we get $k_\star = 14$ and $\epsilon_\star = 2.5$
- For both $D-\mathrm{II}$ and $D-\mathrm{III}$ we get $k_\star = 35$ and $\epsilon_\star = 2.5$

Below we see the Betti-numbers for different combinations of parameters.

![alt text](images/parameters.png "Title")

# Overview

(i) Densely sample a point cloud from target space.

(ii) Train neural networks.

(iii) Compute homology (i.e. Betti numbers at each layer of the neural networks).

For computing the homology, the authors don't use the entirety of the sampled point cloud, but rather a fraction of about $\frac{1}{4}$ of the points, that were used as training sets.

![alt text](images/data_set_sizes.png "Title")

## 3. Results on simulated data

![alt text](images/results_1.png "Title")

The above figure shows the results of the experiment for $D-\mathrm{I}$ for the class $M_{a}$.   
On the bottom we can the the projection on the first two pricipal components.

![alt text](images/results_2.png "Title")

The above figure shows the results of the experiment for $D-\mathrm{2}$ for the class $M_{a}$.

![alt text](images/results_3.png "Title")

The above figure shows the results of the experiment for $D-\mathrm{3}$ for the class $M_{a}$.   

In all experiments, ReLU reduces the betti numbers best. Tanh sometimes even increases the betti numbers.  
Reducing the $\beta_{1}$ numbers in the second experiment seems to be the hardest. Tanh really struggles reducing this to zero.

But in general we can see that non-homeomorphic maps like ReLU or leakyReLU reduce the betti numbers much faster than a homemorphic map like tanh.

Now we will take a look at how the width of the network affects the reduction of betti numbers for the class $M_{a}$.

![alt text](images/results_4.png "Title")

As we can see, the width does not really influence the convergence that much but in general, thinner networks converge a little bit faster but are harder to train. Only the bottleneck network behaves a little bit different. It forces a quick reduction of betti numbers.

At last we will look at different depths of networks for the class $M_{a}$.

![alt text](images/results_5.png "Title")

As you can see, reducing the depth, makes the change in betti numbers much faster but is concentrated in the final layers and the first layers do not seem to play an important role at all for really small networks. The last layers just have to "work harder".  
Also training for such small networks gets really hard.  


## 4. Results on real world data

We will analyze the datasets MNIST Handwritten Digits and HTRU2 High Time-Resolution Universe Survey.  
For these real world datsets, it is not longer possible to train the network near perfectly (a test accuracy of around 95-98% will be achieved).  
The topology fo these datasets is also not know before, so the persistent homology will be computed in every layer.  
Also only one network (10 layers of width 10) will be used.  

### 4.1. MNIST Handwritten Digits

Each of the $70,000$ images in the MNIST handwritten digits data set is an image of size $28 \times 28$-pixels and collectively forms a point cloud on some manifold $M \subseteq \mathbb{R}^{784}$. Computing persistance homology for this space is too hard, so the images will be projected to first 50 principal compontens.

![alt text](images/results2_1.png "Title")

To make a binarfy classification task, we say the class $M_{a}$ is the class of the number $a$ and $M_{b}$ is the class of non-a numbers.

The table below shows the results for $a = 0$ (sum of first 3 betti numbers of $M_{a}$).

![alt text](images/results2_2.png "Title")

The results of this experiment verify the results of the simulated data.  
As you can see, ReLU and leakyReLU reduce betti numbers much faster and tanh even fails to rfeduce the betti numbers to 1 in the last layer.

### 4.2. HTRU2 High Time Resolution Universe Survey

This data set consists of statistics of radio source signals from 17,898 stars, measured during the High Time Resolution Universe Survey (HTRU2) experiment to identify pulsars. For our purpose, it suffices to know that pulsars are stars that produce radio emission measurable on earth. 

The picture below shows the data.

![alt text](images/results2_3.png "Title")

The figure below shows 3 different layers for the tanh network and the ReLU network ($M_{a}$ (red) are the pulsar stars and $M_{b}$ (blue) are the non-pulsar stars).  

![alt text](images/results2_4.png "Title")

![alt text](images/results2_5.png "Title")

As you can see, the results are the same as before. But for this dataset even the ReLU network does not reduce $\beta_{0}$ to 1. Maybe a deeper network would have worked better here.

## 5. Conclusion

We have seen that neural networks indeed work by changing the topology of the given data, reducing its complexits until it becomes linearly seperable.  
We have also seen that using a non-homeomorphic map like ReLU works much better for reducing the betti numbers of the data in contrast to tanh and hece yields better performance for smaller nets.